In [1]:
import pandas as pd

In [2]:
# List of 12 monthly dataset filenames
dataset_filenames = [
    '202201-PV.CSV',
    '202202-PV.CSV',
    '202203-PV.CSV',
    '202204-PV.CSV',
    '202205-PV.CSV',
    '202206-PV.CSV',
    '202207-PV.CSV',
    '202208-PV.CSV',
    '202209-PV.CSV',
    '202210-PV.CSV',
    '202211-PV.CSV',
    '202212-PV.CSV'
]

In [3]:
# List to store the monthly dataframes
monthly_dataframes = []

In [5]:
# Iterate over each dataset filename
for filename in dataset_filenames:
    # Open the monthly dataset
    dataset = pd.read_csv(filename)
    
    # Drop the original headers in the dataset, use the first row as the new headers
    dataset = dataset.rename(columns=dataset.iloc[0]).drop(dataset.index[0]).reset_index(drop=True)
    
    # Drop the last row ('END OF REPORT') in the dataset
    dataset = dataset.iloc[:-1]
    
    # Drop the 1st-4th, 8th and 10th columns (irrelavant columns:'I','ROOFTOP','ACTUAL','2','QI','LASTCHANGED')
    columns_to_drop = dataset.columns[[0, 1, 2, 3, 7, 9]]
    dataset = dataset.drop(columns_to_drop, axis=1)
    
    # Keep all rows with ‘REGIONID’ as: NSW1, QLD1, SA1, VIC1, TAS1; 
    # Reason: all these Region IDs are the sum of other IDs, such as QLDC, QLDS, QLDN, etc
    regions_to_keep = ['NSW1', 'QLD1', 'VIC1', 'TAS1', 'SA1']
    dataset = dataset[dataset['REGIONID'].isin(regions_to_keep)]
    
    # Keep all records with ‘TYPE’ as: ‘MEASUREMENT’; 
    # Reason: ‘MEASUREMENT’ is the actual measured results of ‘POWER’, ‘SATELLITE’ is the estimated results of ‘power’)
    dataset = dataset[dataset['TYPE'] == 'MEASUREMENT'] 
    
    # Clean the 'REGIONID' column: e.g. 'NSW1' to 'NSW'
    dataset['REGIONID'] = dataset['REGIONID'].replace({'NSW1':'NSW','QLD1':'QLD','VIC1':'VIC','SA1':'SA','TAS1':'TAS'})
    
     # Change POWER to numeric
    dataset['POWER'] = pd.to_numeric(dataset['POWER'], errors='coerce')
    
    # Change INTERVAL_DATETIME to datetime
    dataset['INTERVAL_DATETIME'] = pd.to_datetime(dataset['INTERVAL_DATETIME'])
    
    # Add a new column 'Month'
    dataset['Month'] = int(filename[4:6])  # Extract the month from the filename
    
    # Append the monthly dataset to the list
    monthly_dataframes.append(dataset)

In [6]:
# Concatenate the monthly dataframes vertically to get the yearly electricity generation dataframe
yearly_df = pd.concat(monthly_dataframes, ignore_index=True)
yearly_df

,INTERVAL_DATETIME,REGIONID,POWER,TYPE,Month
0,2022-01-01 00:30:00,NSW,0.0,MEASUREMENT,1
1,2022-01-01 00:30:00,QLD,0.0,MEASUREMENT,1
2,2022-01-01 00:30:00,SA,0.0,MEASUREMENT,1
3,2022-01-01 00:30:00,TAS,0.0,MEASUREMENT,1
4,2022-01-01 00:30:00,VIC,0.0,MEASUREMENT,1
...,...,...,...,...,...
90475,2023-01-02 00:00:00,NSW,0.0,MEASUREMENT,12
90476,2023-01-02 00:00:00,QLD,0.0,MEASUREMENT,12
90477,2023-01-02 00:00:00,SA,0.0,MEASUREMENT,12
90478,2023-01-02 00:00:00,TAS,0.0,MEASUREMENT,12


In [13]:
# Delete any rows where the 'INTERVAL_DATETIME' is from 2023
yearly_df = yearly_df[~(yearly_df['INTERVAL_DATETIME'].dt.year == 2023)]
yearly_df

,INTERVAL_DATETIME,REGIONID,POWER,TYPE,Month
0,2022-01-01 00:30:00,NSW,0.0,MEASUREMENT,1
1,2022-01-01 00:30:00,QLD,0.0,MEASUREMENT,1
2,2022-01-01 00:30:00,SA,0.0,MEASUREMENT,1
3,2022-01-01 00:30:00,TAS,0.0,MEASUREMENT,1
4,2022-01-01 00:30:00,VIC,0.0,MEASUREMENT,1
...,...,...,...,...,...
90230,2022-12-31 23:30:00,NSW,0.0,MEASUREMENT,12
90231,2022-12-31 23:30:00,QLD,0.0,MEASUREMENT,12
90232,2022-12-31 23:30:00,SA,0.0,MEASUREMENT,12
90233,2022-12-31 23:30:00,TAS,0.0,MEASUREMENT,12


In [14]:
# 2022 yearly rooftop PV elecrtricity generation aggragation by 'Month' and'REGIONID' 
# Group by 'Month' and 'REGIONID', and sum the 'POWER' - i.e. the power of generator
aggregated_df = yearly_df.groupby(['Month', 'REGIONID'])['POWER'].sum().reset_index()
aggregated_df

,Month,REGIONID,POWER
0,1,NSW,1200837.875
1,1,QLD,1222076.718
2,1,SA,542175.322
3,1,TAS,58631.009
4,1,VIC,955145.349
5,2,NSW,1014204.054
6,2,QLD,1050262.693
7,2,SA,495476.922
8,2,TAS,54659.385
9,2,VIC,834180.974


In [15]:
aggregated_df.isnull().sum()

Month       0
REGIONID    0
POWER       0
dtype: int64

In [18]:
#Add a new column 'Electricity Generation(MWh)'
aggregated_df['Electricity Generation (MWh)'] = round(aggregated_df['POWER'] * (30/60), 2)
aggregated_df

,Month,REGIONID,POWER,Electricity Generation (MWh)
0,1,NSW,1200837.875,600418.94
1,1,QLD,1222076.718,611038.36
2,1,SA,542175.322,271087.66
3,1,TAS,58631.009,29315.50
4,1,VIC,955145.349,477572.67
5,2,NSW,1014204.054,507102.03
6,2,QLD,1050262.693,525131.35
7,2,SA,495476.922,247738.46
8,2,TAS,54659.385,27329.69
9,2,VIC,834180.974,417090.49


In [19]:
#Save the aggregated sheet as the yearly aggregated dataset
aggregated_df.to_csv('aggregatedREGIONID_PV_2022.csv', index=False)